### Leer el archivo JSON usando el "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
language_role_schema = StructType(fields=[
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True),

])

In [0]:
language_role_df = spark.read.schema(language_role_schema).option("multiLine", True).json(
    f"{bronze_folder_path}/{v_file_date}/language_role.json"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
language_role_final_df = language_role_df.withColumnsRenamed({
    'roleId': 'role_id',
    'languageRole': 'language_role',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.languages_roles")
overwrite_partition(language_role_final_df, "movie_silver", "languages_roles", "file_date")


In [0]:
#language_role_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/languages_roles")

#language_role_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages_roles")

language_role_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.languages_roles")

In [0]:
#%fs
#ls /mnt/datamoviehistory/silver/movies_casts

In [0]:
#display(spark.read.parquet("/mnt/datamoviehistory/silver/languages_roles"))

In [0]:
%sql
select file_date, count(1) from movie_silver.languages_roles group by file_date